In [15]:
! pip install 'git+https://github.com/nitkiew2/CMSE495-QSIDE-JUSTFAIR-TOOLBOX.git'

  Cloning https://github.com/nitkiew2/CMSE495-QSIDE-JUSTFAIR-TOOLBOX.git to /private/var/folders/lm/qgrqsck97r52_d1s83h76nkc0000gn/T/pip-req-build-7d7lykom
  Running command git clone -q https://github.com/nitkiew2/CMSE495-QSIDE-JUSTFAIR-TOOLBOX.git /private/var/folders/lm/qgrqsck97r52_d1s83h76nkc0000gn/T/pip-req-build-7d7lykom
  Resolved https://github.com/nitkiew2/CMSE495-QSIDE-JUSTFAIR-TOOLBOX.git to commit 67b5162bddfb836d2c4263f4533fb0b5f72a994f


In [16]:
import JUSTFAIR_Tools as jt
#from JUSTFAIR_Tools import toolbox

In [17]:
jt.toolbox.hello_world()

hello world!


In [18]:
jt.toolbox.matplotlib_test()

NameError: name 'plt' is not defined